## Setup

In [1]:
!pip install datasets -qq
!pip install transformers -qq
!pip install rouge_score evaluate nltk -qq

In [2]:
import torch
import numpy as np
import datasets
import nltk
import evaluate

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from collections import Counter
import matplotlib.pyplot as plt

## Load dataset

In [3]:
# load full xsum dataset
train_dataset, dev_dataset, test_dataset = datasets.load_dataset("xsum",split=['train[30000:100000]', 'validation[3000:10000]', 'test[3000:10000]'])
xsum = datasets.DatasetDict({
    'train': train_dataset,
    'validation': dev_dataset,
    'test': test_dataset,
})

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
xsum

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 70000
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 7000
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 7000
    })
})

In [5]:
xsum["validation"][0]

{'document': '"After eight wonderful years on The X Factor it\'s time for me to move on," he said in a statement.\n"I\'d like to thank ITV, Simon [Cowell], The X Factor family and particularly the viewers, all of whom have been a big part of my life for so long."\nThe 41-year-old took over from Kate Thornton in 2007 as host of the popular talent show.\nThe presenter and broadcaster currently presents a show on Radio 2 and recently completed a 24-hour dance challenge for Comic Relief.\n"I\'d like to wish the team all the best for the future, especially whoever takes over from me," O\'Leary continued.\n"You\'re about to be the conductor on the most brilliant, runaway train in showbiz. Good luck you\'ll love it!"\nThe news of his departure may fuel speculation that O\'Leary is a contender to replace Jeremy Clarkson on the BBC\'s Top Gear.\nBefore Friday\'s announcement, William Hill had O\'Leary as 6-1 to take over following the BBC\'s decision not to renew Clarkson\'s contract earlier th

In [6]:
xsum["validation"].features

{'document': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

## Load pretrained BART and tokenizer

In [7]:
model_name = 'harouzie/bart-base-xsum'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
print(model.config)

BartConfig {
  "_name_or_path": "harouzie/bart-base-xsum",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embedd

In [9]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [10]:
print(f"BART base size: {np.round(model.num_parameters()/1e6, 1)} M parameters")

BART base size: 139.4 M parameters


## Tokenization

In [11]:
max_input_length = 1024
max_target_length = 128

In [12]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["document"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

In [13]:
tokenized_datasets = xsum.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, max_input_length, max_target_length
    ),
    batched=True,
    remove_columns=xsum['train'].column_names,
)

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 70000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7000
    })
})

In [15]:
tokenizer.model_max_length

1024

In [16]:
tokenizer.vocab_size

50265

## Training

### Metrics: ROUGE

In [17]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [18]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Training arguments

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=3,  
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=8,
    gradient_accumulation_steps = 4,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=100,
    save_total_limit=3,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train

In [21]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 70000
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 13125
  Number of trainable parameters = 139420416
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,3.433600
200,3.409800
300,3.434500
400,3.439600
500,3.449500
600,3.477100
700,3.458000
800,3.441900
900,3.458400
1000,3.435400


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Configuration saved in results/checkpoint-500/generation_config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Configuration saved in results/checkpoint-1000/generation_config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Configuration saved in results/checkpoint-1500/generation_config.json
Model weights saved in resul

TrainOutput(global_step=13125, training_loss=3.1864783918108257, metrics={'train_runtime': 18877.698, 'train_samples_per_second': 11.124, 'train_steps_per_second': 0.695, 'total_flos': 1.280446562304e+17, 'train_loss': 3.1864783918108257, 'epoch': 3.0})

In [22]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 7000
  Batch size = 8
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

{'eval_loss': 3.1328420639038086,
 'eval_rouge1': 37.3896,
 'eval_rouge2': 16.406,
 'eval_rougeL': 30.8594,
 'eval_rougeLsum': 30.8619,
 'eval_gen_len': 19.6073,
 'eval_runtime': 656.091,
 'eval_samples_per_second': 10.669,
 'eval_steps_per_second': 1.334,
 'epoch': 3.0}

In [23]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["document"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [24]:
from huggingface_hub import login
login()

In [26]:
model.push_to_hub("harouzie/bart-base-xsum")

Configuration saved in /tmp/tmptux2qowu/config.json
Configuration saved in /tmp/tmptux2qowu/generation_config.json
Model weights saved in /tmp/tmptux2qowu/pytorch_model.bin
Uploading the following files to harouzie/bart-base-xsum: pytorch_model.bin,generation_config.json,config.json


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harouzie/bart-base-xsum/commit/000367a616aa0db963b49c4fd03313425d0d064d', commit_message='Upload BartForConditionalGeneration', commit_description='', oid='000367a616aa0db963b49c4fd03313425d0d064d', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
tokenizer.push_to_hub("harouzie/bart-base-xsum")

tokenizer config file saved in /tmp/tmp_k2f8umg/tokenizer_config.json
Special tokens file saved in /tmp/tmp_k2f8umg/special_tokens_map.json
Uploading the following files to harouzie/bart-base-xsum: merges.txt,special_tokens_map.json,tokenizer.json,tokenizer_config.json,vocab.json


CommitInfo(commit_url='https://huggingface.co/harouzie/bart-base-xsum/commit/4463ab3693f5cf4ee0eaa9a62adc668fb2435c6e', commit_message='Upload tokenizer', commit_description='', oid='4463ab3693f5cf4ee0eaa9a62adc668fb2435c6e', pr_url=None, pr_revision=None, pr_num=None)